<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN User User nDCG </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, user_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_uu = knn.UserUser(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
    return results['ndcg'].mean()


In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[14.936144118881728, 'True', 0.06058445052696416]                                                                      
,                                                                                                                      
[9.025972237858547, 'False', 0.05610346953023822]                                                                      
,                                                                                                                      
[94.37932018515271, 'False', 0.06501205808784875]                                                                      
,                                                                                                                      
[81.12169932030032, 'True', 0.06344050858323212]                                                                       
,                                                                                                                      
[12.229307017308352, 'True', 0.059171277

[85.57600809087417, 'True', 0.06544764727274911]                                                                       
,                                                                                                                      
[19.90171559982041, 'False', 0.06281094411580389]                                                                      
,                                                                                                                      
[37.41498713781847, 'True', 0.06422799847329821]                                                                       
,                                                                                                                      
[21.90624755548812, 'False', 0.06303826182970336]                                                                      
,                                                                                                                      
[95.45998169175718, 'True', 0.0639679030

KeyboardInterrupt: 

In [4]:
array = [[34.48641171617198, 'True', 0.06363094681770082]                     #195.23/59                                                  
,                                                                             #189.26/54                                         
[12.999969966176861, 'True', 0.05855554675791048]                                                                      
,                                                                                                                      
[23.764560449039923, 'True', 0.06287022761474768]                                                                      
,                                                                                                                      
[84.95893573083268, 'True', 0.06434794494209327]                                                                       
,                                                                                                                      
[67.13600131862556, 'True', 0.0643307532595994]                                                                        
,                                                                                                                      
[55.629810817252, 'False', 0.06537577336974791]                                                                        
,                                                                                                                      
[25.2098403036636, 'True', 0.06273760269041265]                                                                        
,                                                                                                                      
[23.643612066886856, 'True', 0.06300387275860929]                                                                      
,                                                                                                                      
[34.62545288099868, 'False', 0.06414662376000363]                                                                      
,                                                                                                                      
[82.59536335637107, 'True', 0.06534235512607836]                                                                       
,                                                                                                                      
[60.58423527439529, 'True', 0.06663534073167325]                                                                       
,                                                                                                                      
[59.282214476624965, 'False', 0.06483800631180377]                                                                     
,                                                                                                                      
[82.0750647818098, 'False', 0.0649577291219745]                                                                        
,                                                                                                                      
[63.22737713818106, 'True', 0.0649668716178559]                                                                        
,                                                                                                                      
[7.435907230388901, 'False', 0.05702322766658289]                                                                      
,                                                                                                                      
[37.68344241972308, 'False', 0.06314129490401126]                                                                      
,                                                                                                                      
[38.55061810424448, 'True', 0.06443074538711892]                                                                       
,                                                                                                                      
[84.88007929936147, 'True', 0.06525043998304324]                                                                       
,                                                                                                                      
[84.30820923196002, 'True', 0.06611426773079356]                                                                       
,                                                                                                                      
[44.96047821501486, 'True', 0.06388931083980046]                                                                       
,                                                                                                                      
[97.28723069815933, 'True', 0.0637239830462041]                                                                        
,                                                                                                                      
[70.89103936794315, 'True', 0.06406278823806587]                                                                       
,                                                                                                                      
[99.622528948638, 'True', 0.06569442205894761]                                                                         
,                                                                                                                      
[74.91409867077047, 'True', 0.06525053229114249]                                                                       
,                                                                                                                      
[92.2879516717536, 'True', 0.06496200268345453]                                                                        
,                                                                                                                      
[49.38558324118756, 'True', 0.06389750540727644]                                                                       
,                                                                                                                      
[75.46077878791102, 'True', 0.06463681048684185]                                                                       
,                                                                                                                      
[53.064203879642676, 'True', 0.06520100407049335]                                                                      
,                                                                                                                      
[91.44031227341146, 'True', 0.06437974783371433]                                                                       
,                                                                                                                      
[44.979825600597096, 'False', 0.06411704705971388]                                                                     
,                                                                                                                      
[61.849644145466435, 'True', 0.06447522070200143]                                                                      
,                                                                                                                      
[74.82032983192916, 'True', 0.06520157802492739]                                                                       
,                                                                                                                      
[1.0892198344727149, 'True', 0.031272201037801754]                                                                     
,                                                                                                                      
[90.79208742315933, 'True', 0.06525732193477478]                                                                       
,                                                                                                                      
[79.42166292761239, 'True', 0.06347495578235259]                                                                       
,                                                                                                                      
[68.22383237769151, 'False', 0.06515036831461427]                                                                      
,                                                                                                                      
[56.93612789204327, 'True', 0.06506269179417465]                                                                       
,                                                                                                                      
[28.262076804058072, 'True', 0.06410081349898242]                                                                      
,                                                                                                                      
[17.132857814132493, 'True', 0.061358224402148255]                                                                     
,                                                                                                                      
[64.93823653688075, 'False', 0.06426268955789395]                                                                      
,                                                                                                                      
[48.38739595499051, 'True', 0.06532316124001508]                                                                       
,                                                                                                                      
[95.55918561477294, 'True', 0.06618050248272973]                                                                       
,                                                                                                                      
[99.5942335747799, 'False', 0.0632117597569286]                                                                        
,                                                                                                                      
[30.6976250900853, 'True', 0.06321442504648077]                                                                        
,                                                                                                                      
[14.157017202908655, 'False', 0.05923467948880352]                                                                     
,                                                                                                                      
[94.35311699232182, 'True', 0.06456246314744786]                                                                       
,                                                                                                                      
[78.88146889257237, 'True', 0.06456625624195493]                                                                       
,                                                                                                                      
[3.231680603013565, 'False', 0.04670929705626586]                                                                      
,                                                                                                                      
[87.60796998095813, 'True', 0.06519754969040518]                                                                       
,                                                                                                                      
[21.45376876119337, 'True', 0.06279814568973736]                                                                       
,                                                                                                                      
[39.80416822871704, 'True', 0.06334455228234333]                                                                       
,                                                                                                                      
[59.066008459227874, 'False', 0.06481292378584422]                                                                     
,                                                                                                                      
[68.25748649402341, 'True', 0.06492113404451424]                                                                       
,                                                                                                                      
[33.592456205465936, 'True', 0.062363119830316754]                                                                     
,                                                                                                                      
[42.78497392129795, 'True', 0.06371305317060932]                                                                       
,                                                                                                                      
[53.56617282764677, 'True', 0.06507557507575461]                                                                       
,                                                                                                                      
[6.5810936627719485, 'False', 0.05461012307404088]                                                                     
,                                                                                                                      
[71.54261363178102, 'True', 0.06460649534667309]                                                                       
,                                                                                                                      
[88.53666442025941, 'True', 0.06450338309178982]
,
[14.936144118881728, 'True', 0.06058445052696416]                                                                      
,                                                                                                                      
[9.025972237858547, 'False', 0.05610346953023822]                                                                      
,                                                                                                                      
[94.37932018515271, 'False', 0.06501205808784875]                                                                      
,                                                                                                                      
[81.12169932030032, 'True', 0.06344050858323212]                                                                       
,                                                                                                                      
[12.229307017308352, 'True', 0.05917127718389019]                                                                      
,                                                                                                                      
[78.91934442862305, 'False', 0.0642486215489324]                                                                       
,                                                                                                                      
[61.982148151935235, 'True', 0.06498351022623189]                                                                      
,                                                                                                                      
[31.144624796833426, 'False', 0.06444193018821058]                                                                     
,                                                                                                                      
[30.712695955419353, 'False', 0.06253787160350138]                                                                     
,                                                                                                                      
[70.85834960024191, 'True', 0.06452548803226654]                                                                       
,                                                                                                                      
[31.105916256460524, 'True', 0.06421415639188036]                                                                      
,                                                                                                                      
[12.5599175035951, 'True', 0.059036627695318734]                                                                       
,                                                                                                                      
[55.51825331176254, 'True', 0.0640464094853088]                                                                        
,                                                                                                                      
[13.262232388889815, 'False', 0.059431439383774286]                                                                    
,                                                                                                                      
[86.81300028433532, 'True', 0.06514396998227713]                                                                       
,                                                                                                                      
[3.257155203773366, 'False', 0.04708780644686353]                                                                      
,                                                                                                                      
[90.01730111587223, 'True', 0.06453453410391477]                                                                       
,                                                                                                                      
[78.00204424939864, 'False', 0.06516452205629625]                                                                      
,                                                                                                                      
[60.31652423087499, 'True', 0.06478345843536747]                                                                       
,                                                                                                                      
[4.608606210384528, 'False', 0.050783619427405176]                                                                     
,                                                                                                                      
[98.95905795621734, 'False', 0.06490251682303812]                                                                      
,                                                                                                                      
[85.14568730074002, 'False', 0.06364320784695694]                                                                      
,                                                                                                                      
[47.815401474996506, 'False', 0.06395085592817754]                                                                     
,                                                                                                                      
[71.23984288047335, 'True', 0.06439223131739892]                                                                       
,                                                                                                                      
[99.42143008904142, 'True', 0.06605600821344812]                                                                       
,                                                                                                                      
[99.76972832828004, 'False', 0.06213854739579937]                                                                      
,                                                                                                                      
[72.33758149899722, 'True', 0.06488294968051378]                                                                       
,                                                                                                                      
[43.40357703661056, 'False', 0.06454828706358728]                                                                      
,                                                                                                                      
[91.78400046105097, 'True', 0.06483412003691562]                                                                       
,                                                                                                                      
[75.93142531282447, 'False', 0.06552877143587925]                                                                      
,                                                                                                                      
[64.0123066516569, 'True', 0.06460552633247134]                                                                        
,                                                                                                                      
[98.93610282888835, 'False', 0.06613993833768936]                                                                      
,                                                                                                                      
[99.92973045542672, 'True', 0.06608813119015608]                                                                       
,                                                                                                                      
[94.47313969173891, 'False', 0.06507909787216769]                                                                      
,                                                                                                                      
[85.57600809087417, 'True', 0.06544764727274911]                                                                       
,                                                                                                                      
[19.90171559982041, 'False', 0.06281094411580389]                                                                      
,                                                                                                                      
[37.41498713781847, 'True', 0.06422799847329821]                                                                       
,                                                                                                                      
[21.90624755548812, 'False', 0.06303826182970336]                                                                      
,                                                                                                                      
[95.45998169175718, 'True', 0.06396790301722569]                                                                       
,                                                                                                                      
[65.62095647068554, 'False', 0.06346513216232814]                                                                      
,                                                                                                                      
[55.79493537192422, 'True', 0.06370612189842184]]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','nDCG']
metric

,neighbours,center,nDCG
0,34.486412,True,0.063631
1,12.999970,True,0.058556
2,23.764560,True,0.062870
3,84.958936,True,0.064348
4,67.136001,True,0.064331
...,...,...,...
95,37.414987,True,0.064228
96,21.906248,False,0.063038
97,95.459982,True,0.063968
98,65.620956,False,0.063465


In [6]:
metric.sort_values(by=['nDCG'],ascending=False).head(10)

,neighbours,center,nDCG
10,60.584235,True,0.066635
41,95.559186,True,0.066181
90,98.936103,False,0.066140
18,84.308209,True,0.066114
91,99.929730,True,0.066088
83,99.421430,True,0.066056
22,99.622529,True,0.065694
88,75.931425,False,0.065529
93,85.576008,True,0.065448
5,55.629811,False,0.065376


In [7]:
metric.to_csv('metric_nDCG.csv', index=False)